In [0]:
%%sh


# aws s3 ls s3://aardvark-prod-dca-data/oss/GAMEIQ_GENRE_USAGE_AGE_GENDER_RELEASE_KPI/version=v2.0.0/range_type=MONTH/
aws s3 ls s3://aardvark-prod-dca-data/oss/GAMEIQ_GENRE_USAGE_AGE_GENDER_RELEASE_KPI/version=v2.0.0/range_type=MONTH/date=2019-01-31/platform=1/metric_name=IDX/

In [0]:

import datetime
import pandas as pd
# spark.sparkContext.addPyFile("s3://b2c-prod-dca-bdp-data/BDP-PROD-APP-INT-QA/bdp/user_data/application_name=aa_data_test_app/application_code/version=latest/code/bdp_resources/usr/lib/spark/python/dependencies.zip")

# from data_lib.api import spark_loader, NarrowToWideParams, convert_narrow_table_to_wide


s3_path = "s3://aardvark-prod-dca-data/oss/GAMEIQ_GENRE_USAGE_AGE_GENDER_RELEASE_KPI/version=v2.0.0/range_type=MONTH/"

spark.read.parquet(s3_path).createOrReplaceTempView("test_game")
# spark.sql("select date, metric_name, count(value), sum(value) from test_game group by date,metric_name order by date,metric_name asc").show(200000)

df = spark.sql("""

select date, metric_name, sum(c_value) as count, sum(s_value) as sum from
    (select date, metric_name, count(value) as c_value, sum(value) as s_value from test_game where genre_id between 2000000 and 2999999 or genre_id  between 2000 and 2999 group by date,metric_name
    union all
    select date, metric_name, count(value) as c_value, sum(value) as s_value from test_game where genre_id between 500000 and 1300000 group by date,metric_name
    union all
    select date, metric_name, count(value) as c_value, sum(value) as s_value from test_game where genre_id  between 2000 and 2999 or genre_id between 2000000 and 2999999 or genre_id between 500000 and 1300000 group by date,metric_name)
group by date, metric_name
order by date, metric_name asc""")
df.show(200000)
df.write.format("parquet").save("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/2022-02-04/genre_level/", mode="append") #append
print("PASS")




# select date, count(1), sum(EST_AUDIENCE_INDEX),sum(EST_AUDIENCE_PERCENTAGE) 
# from 
#     (select date, EST_AUDIENCE_INDEX, EST_AUDIENCE_PERCENTAGE from FACT_USAGE_GENRE_DEMOGRAPHIC_TEST_NEW_V1_CLUSTER_BY_DATE where IQ_TYPE_CODE='game'
#     union all
#     select date, EST_AUDIENCE_INDEX, EST_AUDIENCE_PERCENTAGE from FACT_USAGE_MODIFIER_DEMOGRAPHIC_TEST_NEW_V1_CLUSTER_BY_DATE where tag_key between 500000 and 1300000
#     union all
#     select date, EST_AUDIENCE_INDEX, EST_AUDIENCE_PERCENTAGE from FACT_USAGE_TAG_DEMOGRAPHIC_TEST_NEW_V1_CLUSTER_BY_DATE where tag_key between 2000 and 2999 or tag_key between 2000000 and 2999999)
# group by date
# order by date desc;




# narrow_to_wide_p = NarrowToWideParams("test_game", ["genre_id", "country", "device_type", "age", "gender", "source","date",platform ], "metric_name", ["PCT", "IDX"], "value", None)

# # Then invoke the function with spark session and narrow to wide params
# # You can get the wide table you expected.
# wide_df = convert_narrow_table_to_wide(spark, narrow_to_wide_p)
# wide_df.show(20)


In [0]:

date='2022-02-04'

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/{}/genre_level/".format(date)).coalesce(1).write.csv("s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/{}/genre_level.csv".format(date),header = 'true')


In [0]:
%%sh


DATE='2022-02-04'

rm -rf /tmp/$DATE/
mkdir -p /tmp/$DATE/

aws s3 cp --recursive s3://b2c-prod-data-pipeline-qa/aa.gwang.s6usage/adhoc/$DATE/genre_level.csv/ /tmp/$DATE/genre_level.csv/



echo "=======retention_week========="
cat /tmp/$DATE/genre_level.csv/*.csv

In [0]:
%%sh
